In [0]:
%sql

-- Drop the function if it already exists to allow recreation during development
DROP FUNCTION IF EXISTS udf_get_workdays_bc;

-- Create the SQL UDF
CREATE FUNCTION udf_get_workdays_bc(
    startDate DATE,
    endDate DATE,
    bcHolidays ARRAY<DATE> DEFAULT ARRAY() -- Optional parameter, defaults to an empty array
)
RETURNS INT
RETURN (
    CASE
        WHEN startDate IS NULL OR endDate IS NULL THEN 0
        WHEN endDate < startDate THEN 0
        ELSE
            REDUCE(
                -- 1. Generate a sequence of all dates between startDate and endDate
                SEQUENCE(startDate, endDate, INTERVAL 1 DAY),
                -- 2. Initialize the accumulator (workday count) to 0
                0,
                -- 3. Lambda function to process each date in the sequence
                (acc, current_date) -> acc + CASE
                                            -- Check if it's a weekend (Sunday=1, Saturday=7 in DAYOFWEEK)
                                            WHEN DAYOFWEEK(current_date) IN (1, 7) THEN 0
                                            -- Check if the current_date is in the provided bcHolidays array
                                            WHEN ARRAY_CONTAINS(bcHolidays, current_date) THEN 0
                                            -- If not a weekend and not a holiday, it's a workday
                                            ELSE 1
                                        END
            )
    END
);

In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
from pyspark.sql.functions import lit, col

tablename = "factRequestCustomCalcFields"
runcycleid = etl_helpers.start_run_cycle(tablename)
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:

    query = f"""delete from factrequestcustomcalcfields where sourceoftruth = 'FOIMOD' """;

    df = spark.sql(query)
    df.show()

    query = f"""

        select 
        distinct
        *
        , case when requeststatuslabel = 'closed' then pageflagcount else 0 end as noofpagesreleased
        , TRY_CAST(onholdbusinessdays.total_business_days_on_hold AS INTEGER) AS onholddays
        , TRY_CAST(onholdotherbusinessdays.total_business_days_on_hold AS INTEGER) AS onholdotherdays
        ,
        TRY_CAST(
            GREATEST(
                COALESCE(noneonholdcloseddays.total_none_hold_closed_days, 0), 
                0
            ) AS INTEGER
        ) AS processeddays
        ,
        TRY_CAST(
            CASE
                WHEN sq2.requeststatuslabel = 'closed' AND TRY_CAST(finalclosedate.closedate AS DATE) IS NOT NULL AND sq2.duedate IS NOT NULL AND sq2.duedate >= TRY_TO_TIMESTAMP(finalclosedate.closedate, 'yyyy-MM-dd HH:mm:ss') THEN udf_get_workdays_bc(
                    TRY_CAST(finalclosedate.closedate AS DATE), -- Start Date
                    TRY_CAST(sq2.duedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays)
                )
                WHEN sq2.requeststatuslabel != 'closed' AND sq2.duedate IS NOT NULL AND sq2.duedate >= CURRENT_DATE() THEN udf_get_workdays_bc(
                    CURRENT_DATE(), -- Start Date
                    TRY_CAST(sq2.duedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays)
                )
            ELSE
                0
            END
            AS INTEGER
        ) AS remainingdays
        ,
        CASE
            WHEN sq2.requeststatusid IS NOT NULL AND sq2.requeststatusid in (11, 19, 22) THEN 1
            WHEN sq2.requeststatuslabel = 'closed' AND TRY_CAST(finalclosedate.closedate AS DATE) IS NOT NULL AND sq2.duedate IS NOT NULL AND sq2.duedate >= TRY_TO_TIMESTAMP(finalclosedate.closedate, 'yyyy-MM-dd HH:mm:ss') THEN 1
            WHEN sq2.requeststatuslabel = 'closed' AND TRY_CAST(finalclosedate.closedate AS DATE) IS NOT NULL AND sq2.duedate IS NOT NULL AND sq2.duedate < TRY_TO_TIMESTAMP(finalclosedate.closedate, 'yyyy-MM-dd HH:mm:ss') THEN 0
            WHEN sq2.duedate IS NOT NULL AND sq2.duedate >= CURRENT_DATE() THEN 1
            WHEN sq2.duedate IS NOT NULL AND sq2.duedate < CURRENT_DATE() THEN 0
        ELSE
            1
        END AS countontime
        ,
        CASE
            WHEN sq2.requeststatusid IS NOT NULL AND sq2.requeststatusid in (11, 19, 22) THEN 0
            WHEN sq2.requeststatuslabel = 'closed' AND TRY_CAST(finalclosedate.closedate AS DATE) IS NOT NULL AND sq2.duedate IS NOT NULL AND sq2.duedate < TRY_TO_TIMESTAMP(finalclosedate.closedate, 'yyyy-MM-dd HH:mm:ss') THEN 1
            WHEN sq2.requeststatuslabel = 'closed' AND TRY_CAST(finalclosedate.closedate AS DATE) IS NOT NULL AND sq2.duedate IS NOT NULL AND sq2.duedate >= TRY_TO_TIMESTAMP(finalclosedate.closedate, 'yyyy-MM-dd HH:mm:ss') THEN 0
            WHEN sq2.duedate IS NOT NULL AND sq2.duedate < CURRENT_DATE() THEN 1
            WHEN sq2.duedate IS NOT NULL AND sq2.duedate >= CURRENT_DATE() THEN 0
        ELSE
            0
        END AS countoverdue
        ,
        TRY_CAST(
            CASE
                WHEN sq2.requeststatusid IS NOT NULL AND sq2.requeststatusid in (11, 19, 22) THEN 0
                WHEN sq2.requeststatuslabel = 'closed' AND TRY_CAST(finalclosedate.closedate AS DATE) IS NOT NULL AND sq2.duedate IS NOT NULL AND TRY_CAST(finalclosedate.closedate AS DATE) > sq2.duedate THEN udf_get_workdays_bc(
                    TRY_CAST(sq2.duedate AS DATE), -- Start Date
                    TRY_CAST(finalclosedate.closedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays)
                )
                WHEN sq2.requeststatuslabel != 'closed' AND sq2.duedate IS NOT NULL AND CURRENT_DATE() > sq2.duedate THEN udf_get_workdays_bc(
                    TRY_CAST(sq2.duedate AS DATE), -- Start Date
                    CURRENT_DATE(), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays)
                )
            ELSE
                0
            END
            AS INTEGER
        ) AS daysoverdue
        , watchers.watcher_list AS secondaryusers
        , TRY_CAST(records.noofdoc AS INTEGER) AS noofdocdelivered

        from 

        (
            SELECT *
            FROM (
                SELECT 
                duedate as firstduedate,
                foirequest_id,
                foiministryrequestid,
                --*,
                ROW_NUMBER() OVER (
                    PARTITION BY foiministryrequestid 
                    ORDER BY created_at asc
                ) AS rn
                FROM foi_mod.foiministryrequests
            ) sub
            WHERE rn = 1
        ) sq
        
        join

        (
            SELECT *
            FROM (
                SELECT 
                duedate,
                foiministryrequestid,
                version,
                requeststatuslabel,
                requeststatusid,
                created_at,
                axisrequestid,
                --*,
                ROW_NUMBER() OVER (
                    PARTITION BY foiministryrequestid 
                    ORDER BY created_at desc
                ) AS rn
                FROM foi_mod.foiministryrequests
                -- where created_at > '2026-01-01'
                -- where axisrequestid = 'CFD-2026-60130'
            ) sub
            WHERE rn = 1
        ) sq2 on sq2.foiministryrequestid = sq.foiministryrequestid

        left join

        (
            SELECT
                foiministryrequestid,
                createdby,
                TRY_CAST(closedate AS DATE) AS closedate
            FROM foi_mod.foiministryrequests
            WHERE closedate IS NOT NULL AND closedate != 'NULL'
            QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version DESC) = 1
        ) finalclosedate on finalclosedate.foiministryrequestid = sq2.foiministryrequestid

        left join

        (
            SELECT
                foiministryrequestid,
                SUM(days_in_status) AS total_business_days_on_hold
            FROM (
                SELECT
                    foiministryrequestid,
                    requeststatusid,
                    MIN(created_at) AS status_start_date,
                    -- The end date of a status period is the created_at of the next different status version
                    -- Or CURRENT_DATE() if it's the latest status for that foiministryrequestid
                    COALESCE(
                      LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                      CURRENT_DATE() -- Use CURRENT_DATE() to include time in current status
                    ) AS status_end_date,
                    udf_get_workdays_bc(
                      MIN(created_at), -- On hold start date
                      COALESCE(
                        LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                        CURRENT_DATE()
                      ), -- On hold start date
                      (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays) -- Pass the array of BC holidays
                    )-1 AS days_in_status
                FROM (
                    SELECT
                      foiministryrequestid,
                      version,
                      requeststatusid,
                      created_at,
                      -- Create a running sum to identify consecutive status blocks
                      SUM(CASE WHEN LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) != requeststatusid OR LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) IS NULL THEN 1 ELSE 0 END)
                        OVER (PARTITION BY foiministryrequestid ORDER BY version) AS status_group_id
                    FROM
                      foi_mod.foiministryrequests
                ) AS sub_inner
                GROUP BY
                    foiministryrequestid,
                    requeststatusid,
                    status_group_id
            ) AS sub_outer
            WHERE
                requeststatusid in (11, 19) --'On Hold'
            GROUP BY
                foiministryrequestid
        ) onholdbusinessdays ON onholdbusinessdays.foiministryrequestid = sq.foiministryrequestid

        left join

        (
            SELECT
                foiministryrequestid,
                SUM(days_in_status) AS total_business_days_on_hold
            FROM (
                SELECT
                    foiministryrequestid,
                    requeststatusid,
                    MIN(created_at) AS status_start_date,
                    -- The end date of a status period is the created_at of the next different status version
                    -- Or CURRENT_DATE() if it's the latest status for that foiministryrequestid
                    COALESCE(
                      LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                      CURRENT_DATE() -- Use CURRENT_DATE() to include time in current status
                    ) AS status_end_date,
                    udf_get_workdays_bc(
                      MIN(created_at), -- On hold start date
                      COALESCE(
                        LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                        CURRENT_DATE()
                      ), -- On hold start date
                      (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays) -- Pass the array of BC holidays
                    )-1 AS days_in_status
                FROM (
                    SELECT
                      foiministryrequestid,
                      version,
                      requeststatusid,
                      created_at,
                      -- Create a running sum to identify consecutive status blocks
                      SUM(CASE WHEN LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) != requeststatusid OR LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) IS NULL THEN 1 ELSE 0 END)
                        OVER (PARTITION BY foiministryrequestid ORDER BY version) AS status_group_id
                    FROM
                      foi_mod.foiministryrequests
                ) AS sub_inner
                GROUP BY
                    foiministryrequestid,
                    requeststatusid,
                    status_group_id
            ) AS sub_outer
            WHERE
                requeststatusid in (22, 20)   --'On Hold - Other' & 'Section 5 Pending (Personal)'
            GROUP BY
                foiministryrequestid
        ) onholdotherbusinessdays ON onholdotherbusinessdays.foiministryrequestid = sq.foiministryrequestid

        left join

        (
            WITH LatestStarts AS (
                -- Get the most recent startdate for each request
                SELECT 
                    foiministryrequestid, 
                    MAX(startdate) AS latest_startdate
                FROM foi_mod.foiministryrequests
                GROUP BY foiministryrequestid
            )
            SELECT
                sub_outer.foiministryrequestid,
                SUM(sub_outer.days_in_status) AS total_none_hold_closed_days
            FROM (
                -- Main status history logic
                SELECT
                    foiministryrequestid,
                    requeststatusid,
                    MIN(created_at) AS status_start_date,
                    -- The end date of a status period is the created_at of the next different status version
                    -- Or CURRENT_DATE() if it's the latest status for that foiministryrequestid
                    COALESCE(
                        LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                        CURRENT_DATE() -- Use CURRENT_DATE() to include time in current status
                    ) AS status_end_date,
                    udf_get_workdays_bc(
                        MIN(created_at), -- On hold start date
                        COALESCE(
                            LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                            CURRENT_DATE()
                        ), -- On hold start date
                        (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays) -- Pass the array of BC holidays
                    )-1 AS days_in_status
                FROM (
                    SELECT
                        foiministryrequestid,
                        version,
                        requeststatusid,
                        created_at,
                        -- Create a running sum to identify consecutive status blocks
                        SUM(CASE
                            WHEN LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) != requeststatusid
                            OR LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) IS NULL
                            THEN 1 ELSE 0 END)
                        OVER (PARTITION BY foiministryrequestid ORDER BY version) AS status_group_id
                    FROM
                        foi_mod.foiministryrequests
                ) AS sub_inner
                GROUP BY
                    foiministryrequestid,
                    requeststatusid,
                    status_group_id

                UNION ALL

                -- processed days before open (before created in foiministryrequests table)
                SELECT
                    foiministryrequestid,
                    0 AS requeststatusid,
                    startdate AS status_start_date,
                    created_at AS status_end_date,
                    udf_get_workdays_bc(
                        startdate,
                        created_at,
                        (SELECT COLLECT_LIST(holiday_date) FROM default.bc_gov_holidays) -- Pass the array of BC holidays
                    )-1 AS days_in_status
                FROM foi_mod.foiministryrequests
                WHERE version = 1
            ) AS sub_outer
            JOIN LatestStarts ls ON sub_outer.foiministryrequestid = ls.foiministryrequestid
            WHERE
                sub_outer.requeststatusid not in (3, 11, 19, 20, 22)   --'None On Hold/Close'
                -- Strictly filter for status periods starting ON or AFTER the latest startdate
                AND sub_outer.status_start_date >= ls.latest_startdate
            GROUP BY
                sub_outer.foiministryrequestid
        ) noneonholdcloseddays ON noneonholdcloseddays.foiministryrequestid = sq.foiministryrequestid

        left join

        (SELECT *
                FROM (
                    SELECT 
                    created_at as firstonholddate,
                    foiministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY foiministryrequestid 
                            ORDER BY created_at asc
                        ) AS rn
                    FROM foi_mod.foiministryrequests
                    where requeststatuslabel = 'onhold'
                ) sub
                WHERE rn = 1) sq4 on sq4.foiministryrequestid = sq.foiministryrequestid

        left join

        (-- Step 1: Parse pageflag as JSON array of structs and explode it
                WITH exploded_flags AS (
                SELECT
                    documentid,
                    EXPLODE(from_json(pageflag, 'ARRAY<STRUCT<flagid: INT, page: INT, programareaid: ARRAY<INT>, other: ARRAY<STRING>>>')) AS flag
                FROM docreviewer.DocumentPageflags
                ),

                -- Step 2: Select the needed fields
                parsed_flags AS (
                SELECT
                    documentid,
                    flag.flagid
                FROM exploded_flags
                WHERE flag.flagid IS NOT NULL  -- only count flags that have a flagid
                )

                -- Step 3: Aggregate
                SELECT
                -- pf.documentid,
                -- pf.flagid,
                COUNT(*) AS pageflagcount,
                dpf.foiministryrequestid,
                'Y' AS isactive
                FROM parsed_flags pf
                JOIN docreviewer.DocumentPageflags dpf
                ON pf.documentid = dpf.documentid
                GROUP BY dpf.foiministryrequestid) sq6 on sq6.foiministryrequestid = sq.foiministryrequestid

        left join

        (SELECT *
                FROM (
                    SELECT 
                    ministryrequestid,
                    get_json_object(feedata, '$.estimatedelectronicpages') as electronicpageestimate,
                    get_json_object(feedata, '$.amountpaid') as feepaidamount,
                    get_json_object(feedata, '$.refundamount') as refundamount,
                    CAST(get_json_object(feedata, '$.refundamount') AS DECIMAL(10, 2)) -
                      CAST(get_json_object(feedata, '$.balanceremaining') AS DECIMAL(10, 2)) AS prepaymentamount,
                    get_json_object(feedata, '$.esetimatedhardcopypages') as physicalpageestimate,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY ministryrequestid 
                            ORDER BY created_at desc
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees
                ) sub
                WHERE rn = 1) sq7 on sq7.ministryrequestid = sq.foiministryrequestid

        left join

        (select statusid, investigator, r.name as reason, rt.name as reviewtype, isjudicialreview, oipcno, foiministryrequest_id, foiministryrequestversion_id
        from 
        (
            SELECT
                foiministryrequest_id,
                foiministryrequestversion_id,
                oipcno,
                isjudicialreview,
                investigator,
                reviewtypeid,
                reasonid,
                statusid,
                created_at
            FROM foi_mod.foirequestoipc
            QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequest_id ORDER BY foiministryrequestversion_id DESC) = 1
        ) o
        join foi_mod.oipcreasons r on r.reasonid = o.reasonid
        join foi_mod.oipcreviewtypes rt on rt.reviewtypeid = o.reviewtypeid
        ) sq8 on sq8.foiministryrequest_id = sq2.foiministryrequestid and sq8.foiministryrequestversion_id = sq2.version

        -- oi table not in prod yet
        -- (select iaorationale, ps.name
        -- from foi_mod.foiopeninformationrequests oi
        -- join openinfopublicationstatuses ps on ps.oipublicationstatus_id = oi.oipublicationstatus_id)

        left join

        (select foiministryrequestid, foiministryrequestversion, name as subject from foi_mod.foiministryrequestsubjectcodes msc
        join foi_mod.subjectcodes sc on msc.subjectcodeid = sc.subjectcodeid) sq9 on sq9.foiministryrequestid = sq2.foiministryrequestid and sq9.foiministryrequestversion = sq2.version

        left join

        (select amountpaid, refundamount as applicationfeerefunded, foiministryrequestid
        from foi_mod.foirequestapplicationfees af
        join foi_mod.foirequests r on r.foirawrequestid = af.rawrequestid
        join foi_mod.foiministryrequests mr on r.foirequestid = mr.foirequest_id) sq10 on sq10.foiministryrequestid = sq.foiministryrequestid

        left join
        (select sum(pagecount) as dedupepagecount, foiministryrequestid from docreviewer.documents d1
        join 
        (select rank1hash, min(d.documentid) as docid  from docreviewer.documenthashcodes dhc
        join docreviewer.documents d on d.documentid = dhc.documentid
        group by rank1hash) sq on sq.docid = d1.documentid
        group by foiministryrequestid) sq12 on sq12.foiministryrequestid = sq.foiministryrequestid

        left join 
        
        (
            SELECT
                ministryrequestid,
                ARRAY_JOIN(COLLECT_SET(watchedby), ', ') AS watcher_list
            FROM foi_mod.foirequestwatchers
            WHERE isactive = 't'
            GROUP BY ministryrequestid
        ) watchers ON watchers.ministryrequestid = sq.foiministryrequestid

        left join

        (
            SELECT
                ministryrequestid,
                ministryrequestversion,
                count(recordid) AS noofdoc
            FROM foi_mod.foirequestrecords
            WHERE isactive = 't'
            GROUP BY ministryrequestid, ministryrequestversion
        ) records ON records.ministryrequestid = sq2.foiministryrequestid AND records.ministryrequestversion = sq2.version

        left join
        
        (
            SELECT
                oi.foiministryrequest_id as foiministryrequestid,
                oistatus.name AS publication,
                oie.name AS publicationreason
            FROM
                (
                    SELECT
                        foiministryrequest_id,
                        oiexemption_id,
                        oipublicationstatus_id
                    FROM
                        foi_mod.foiopeninformationrequests
                    QUALIFY ROW_NUMBER() OVER (
                        PARTITION BY foiministryrequest_id
                        ORDER BY version DESC
                    ) = 1
                ) AS oi
            LEFT JOIN foi_mod.openinfopublicationstatuses AS oistatus
                ON oistatus.oipublicationstatusid = oi.oipublicationstatus_id
            LEFT JOIN foi_mod.openinformationexemptions AS oie
                ON oie.oiexemptionid = oi.oiexemption_id
        ) oiinfo ON oiinfo.foiministryrequestid = sq.foiministryrequestid

        """

    print(query)

    df = spark.sql(query)
    print(df.count())
    df.show()


    # order of columns here is important!
    df_mapped = df.selectExpr( 
        "foirequest_id as foirequestid",
        f"{runcycleid} as runcycleid",
        "'' as identityverification",
        "'' as previousfoirequest",
        "amountpaid as applicationfeepaid",
        "applicationfeerefunded as applicationfeerefunded",
        "'' as applicationfeetransferred",
        "'' as linkedrequests",
        "double(feepaidamount) as feepaidamount",
        "double(refundamount) as refundamount",
        "prepaymentamount as prepaymentamount",
        "physicalpageestimate as physicalpageestimate",
        "electronicpageestimate as electronicpageestimate",
        "statusid as customfieldstatus",
        "secondaryusers as secondaryusers",
        "'' as applicantfilereference",
        "'' as coordinatednrresponsereqd",
        "investigator AS portfolioofficer",
        "reason AS reason",
        "reviewtype AS reviewtype",
        "isjudicialreview AS judicialreview",
        "oipcno AS oipcno",
        "publicationreason AS publicationreason",
        "publication AS publication",
        "'' AS crossgovtno",
        "subject as subject",
        "created_at AS currentactivitydate",
        "noofdocdelivered AS noofdocdelivered",
        "lastonholddate - firstonholddate AS onholddays",
        "created_at <= duedate as countontime",
        "created_at > duedate as countoverdue",
        "daysoverdue AS daysoverdue",
        "daysoverdue as passduedays",
        "pageflagcount as noofpagesreviewed",
        "pageflagcount as noofpagesreleased",
        "dedupepagecount AS noofpagesdeduplicated",
        "'FOIMOD' as sourceoftruth",
        "onholdotherdays as onholdotherdays",
        "processeddays as processeddays",
        "remainingdays as remainingdays",
    )
    df_mapped.show()    
    df_mapped.printSchema()
    df_mapped.write.format("delta").mode("append").option("mergeSchema", "false").insertInto(tablename)  
    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
    raise Exception("notebook failed") from e
except Exception as e:    
    if (str(e) == "no changes for today"):
        # print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")
        raise Exception("notebook failed") from e